# 0. Settings

In [ ]:
# Change these

locality = "nc-guilford"
verbose = True
clear_checkpoints = False

# 1. Basic setup

In [ ]:
from init_notebooks import setup_environment
setup_environment()

In [ ]:
# import a bunch of stuff
import os
import pickle
import pandas as pd
from openavmkit.pipeline import (
    NotebookState, 
    set_locality,
    examine_df,
    load_dataframes,
    load_settings,
    process_data,
    process_sales
)
from openavmkit.data import (
    sup_tag_model_groups
)
from openavmkit.checkpoint import (
    from_checkpoint,
    delete_checkpoints
)

In [ ]:
if 'inited' not in globals():
    nbs: NotebookState = None
    inited = True
nbs = set_locality(nbs, locality)

In [ ]:
if clear_checkpoints:
    delete_checkpoints("1-assemble")

# 2. Load & process data

In [ ]:
settings = load_settings()

In [ ]:
# load all of our data
dataframes = from_checkpoint("1-assemble-01-load_dataframes", load_dataframes,
    {
        "settings":settings,
        "verbose":verbose
    }
)

In [ ]:
# process all of our data
sales_univ_pair = from_checkpoint("1-assemble-02-process_data", process_data,
    {
        "dataframes":dataframes, 
        "settings":settings, 
        "verbose":verbose
    }
)

In [ ]:
# process all of our sales
sales_univ_pair = from_checkpoint("1-assemble-03-process_sales", process_sales,
    {
        "sup": sales_univ_pair, 
        "settings": settings, 
        "verbose": verbose
    }
)

# 3. Tag modeling groups

In [ ]:
sales_univ_pair = from_checkpoint("1-assemble-04-tag_modeling_groups", sup_tag_model_groups,
    {
        "sup": sales_univ_pair, 
        "settings": settings, 
        "verbose": verbose
    }
)

# 4. Write out results

In [ ]:
with open (f"out/sales_univ.pickle", "wb") as file:
    pickle.dump(sales_univ_pair, file)

In [ ]:
os.makedirs("out/look", exist_ok=True)
sales_univ_pair["universe"].to_parquet("out/look/1-assemble-universe.parquet")
sales_univ_pair["sales"].to_parquet("out/look/1-assemble-sales.parquet")